In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
mydir = 'data_dir'
os.listdir(mydir)

['holidays_events.csv',
 'oil.csv',
 'sample_submission.csv',
 'stores.csv',
 'test.csv',
 'train.csv',
 'transactions.csv']

In [3]:
train_data = pd.read_csv('data_dir/train.csv')
train_data.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [4]:
train_data.shape

(3000888, 6)

In [5]:
he_df   = pd.read_csv('data_dir/holidays_events.csv')    
he_df = he_df[['date','type']].drop_duplicates()
he_df = he_df.rename(columns = {'type':'holiday_type'})
he_df.head()

,date,holiday_type
0,2012-03-02,Holiday
1,2012-04-01,Holiday
2,2012-04-12,Holiday
3,2012-04-14,Holiday
4,2012-04-21,Holiday


In [7]:
train_data = pd.merge(train_data, he_df[he_df['holiday_type'] == 'Holiday'][['date','holiday_type']], on = 'date', how= 'left')

In [9]:
train_data.shape

(3000888, 7)

In [10]:
oil_df  = pd.read_csv('data_dir/oil.csv')
oil_df  = oil_df.drop_duplicates()
oil_df  = oil_df.dropna(subset = 'dcoilwtico')
oil_df.head()

,date,dcoilwtico
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21


In [11]:
train_data = pd.merge(train_data, oil_df, on = 'date', how= 'left')

In [12]:
train_data.head()

,id,date,store_nbr,family,sales,onpromotion,holiday_type,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Holiday,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Holiday,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Holiday,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Holiday,NaN


In [13]:
stores_df  = pd.read_csv('data_dir/stores.csv')
stores_df  = stores_df.drop_duplicates()
stores_df = stores_df.dropna()
stores_df = stores_df.rename(columns = {'type':'store_type'})
stores_df.head()

,store_nbr,city,state,store_type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [14]:
train_data = pd.merge(train_data, stores_df, on = 'store_nbr', how= 'left')

In [15]:
train_data.head()

,id,date,store_nbr,family,sales,onpromotion,holiday_type,dcoilwtico,city,state,store_type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,NaN,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,Holiday,NaN,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,Holiday,NaN,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,Holiday,NaN,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,Holiday,NaN,Quito,Pichincha,D,13


In [17]:
txn_df  = pd.read_csv('data_dir/transactions.csv')
txn_df  = txn_df.drop_duplicates()
txn_df  = txn_df.dropna()
txn_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [18]:
train_data = pd.merge(train_data, txn_df, on=['date','store_nbr'],how='left')


In [22]:
train_data.to_csv('data_dir/preprocessed_train_data.csv')